# Loss db set up

This notebook compares the count of buildings before and after db set up to ensure no significant losses occured. 

Losses maybe happen if a dataset uses slightly different boundaries than the administrative boundaries used for the spatial joins (NUTS).

The NUTS file available has a relatively coarse resolution, which causes issues in particular in countries with high amounts of cost lines or borders.

We compare stat files produced after the parsing stage to overview files produced after db set up at the country level.

We consider a loss up to 1% acceptable, and up to 4% for countries with high amount of coast line (e.g. Norway or Croatia). 


In [2]:
import geopandas as gpd
import pandas as pd
import glob 
import os

root = '/home/nmd/Cluster'

# Microsoft

In [11]:
parsing_paths = glob.glob(os.path.join(root,'stats/1-parsing','*msft_stat.csv'))

In [96]:
df_parsing = pd.DataFrame()
for path in parsing_paths:
    df_parsing = pd.concat([df_parsing,pd.read_csv(path)[['dataset_name','n_bldgs']]])
df_parsing['dataset_name'] = [x.split('-')[0].lower() for x in df_parsing['dataset_name']]

In [97]:
overview_paths = glob.glob(os.path.join(root,'stats/2-db-set-up/overview/v1','msft*.csv'))

In [98]:
df_overview = pd.DataFrame()
for path in overview_paths:
    tmp = pd.read_csv(path)[['n_bldgs']]
    tmp = pd.DataFrame([{'dataset_name':os.path.split(path)[-1].split('_')[1],
                         'n_bldgs':tmp.n_bldgs.sum()
                         }])
    df_overview = pd.concat([df_overview,tmp])

In [99]:
df = pd.merge(df_parsing,df_overview,on='dataset_name',suffixes=('_parsing','_db set up'))

In [100]:
df['loss_abs'] = df['n_bldgs_parsing'] - df['n_bldgs_db set up']
df['loss_per'] = round(df['loss_abs'] / df['n_bldgs_parsing'],2)

In [103]:
df.sort_values(by='dataset_name').reset_index(drop=True)

,dataset_name,n_bldgs_parsing,n_bldgs_db set up,loss_abs,loss_per
0,austria,3721255,3714329,6926,0.00
1,belgium,4557540,4551763,5777,0.00
2,bulgaria,4027449,4026100,1349,0.00
3,croatia,2832178,2800971,31207,0.01
4,cyprus,789636,788362,1274,0.00
5,czechia,3412978,3411551,1427,0.00
6,denmark,3547677,3538250,9427,0.00
7,estonia,736499,734276,2223,0.00
8,finland,4593802,4543299,50503,0.01
9,france,26019438,26001484,17954,0.00


# OpenStreetMap

In [9]:
parsing_paths = glob.glob(os.path.join(root,'stats/1-parsing','*osm_stat.csv'))

In [10]:
df_parsing = pd.DataFrame()
for path in parsing_paths:
    df_parsing = pd.concat([df_parsing,pd.read_csv(path)[['dataset_name','n_bldgs']]])

In [11]:
df_parsing = df_parsing[~df_parsing.dataset_name.isin(['Berlin.osm','Bremen.osm'])]

In [12]:
deu = ['baden-wuerttemberg-latest.osm',
'bayern-latest.osm',
'Berlin.osm',
'brandenburg-latest.osm',
'Bremen.osm',
'Hamburg.osm',
'hessen-latest.osm',
'mecklenburg-vorpommern-latest.osm',
'niedersachsen-latest.osm',
'nordrhein-westfalen-latest.osm',
'rheinland-pfalz-latest.osm',
'saarland-latest.osm',
'sachsen-anhalt-latest.osm',
'sachsen-latest.osm',
'schleswig-holstein-latest.osm',
'thueringen-latest.osm']

In [13]:
fr = ['alsace-latest.osm',
'aquitaine-latest.osm',
'auvergne-latest.osm',
'basse-normandie-latest.osm',
'bourgogne-latest.osm',
'bretagne-latest.osm',
'centre-latest.osm',
'champagne-ardenne-latest.osm',
'corse-latest.osm',
'franche-comte-latest.osm',
'haute-normandie-latest.osm',
'picardie-latest.osm',
'lorraine-latest.osm',
'rhone-alpes-latest.osm',
'provence-alpes-cote-d-azur-latest.osm',
'ile-de-france-latest.osm',
'languedoc-roussillon-latest.osm',
'limousin-latest.osm',
'midi-pyrenees-latest.osm',
'nord-pas-de-calais-latest.osm',
'pays-de-la-loire-latest.osm',
'poitou-charentes-latest.osm']

In [14]:
df_parsing['dataset_name'] = ['germany' if x in deu else x for x in df_parsing['dataset_name']]
df_parsing['dataset_name'] = ['france' if x in fr else x for x in df_parsing['dataset_name']]

In [15]:
df_parsing_fr_de = df_parsing[df_parsing.dataset_name.isin(['france','germany'])]
df_parsing = df_parsing[~df_parsing.dataset_name.isin(['france','germany'])]
df_parsing_fr_de = df_parsing_fr_de.groupby('dataset_name')['n_bldgs'].sum().reset_index()

In [16]:
df_parsing['dataset_name'] = [x.split('-')[0].lower() for x in df_parsing['dataset_name']]
df_parsing = pd.concat([df_parsing,df_parsing_fr_de])

overview_paths = glob.glob(os.path.join(root,'stats/2-db-set-up/overview/v1','osm*.csv'))
df_overview = pd.DataFrame()
for path in overview_paths:
    tmp = pd.read_csv(path)[['n_bldgs']]
    tmp = pd.DataFrame([{'dataset_name':os.path.split(path)[-1].split('_')[1],
                         'n_bldgs':tmp.n_bldgs.sum()
                         }])
    df_overview = pd.concat([df_overview,tmp])

df = pd.merge(df_parsing,df_overview,on='dataset_name',suffixes=('_parsing','_db set up'),how='left')

In [17]:
uk_overview = pd.read_csv('/home/nmd/Cluster/stats/2-db-set-up/overview/v1/osm_uk_overview.csv')
northern_ireland = uk_overview[uk_overview.NUTS3_ID.isin([x for x in uk_overview.NUTS3_ID if 'UKN' in x])].n_bldgs.sum()

In [18]:
# northern ireland included in ireland file
df.at[9,'n_bldgs_parsing'] = df.at[9,'n_bldgs_parsing'] - northern_ireland 

In [19]:
df['loss_abs'] = df['n_bldgs_parsing'] - df['n_bldgs_db set up']
df['loss_per'] = round(df['loss_abs'] / df['n_bldgs_parsing'],2)

In [20]:
df.sort_values(by='dataset_name').reset_index(drop=True)

,dataset_name,n_bldgs_parsing,n_bldgs_db set up,loss_abs,loss_per
0,austria,4235874,4212547,23327,0.01
1,belgium,6530279,6501847,28432,0.00
2,bulgaria,669828,668457,1371,0.00
3,croatia,1149108,1120341,28767,0.03
4,cyprus,216983,216711,272,0.00
5,czechia,4948416,4884697,63719,0.01
6,denmark,3701268,3691822,9446,0.00
7,estonia,892258,888165,4093,0.00
8,finland,2770832,2721889,48943,0.02
9,france,49192283,48948205,244078,0.00


# Government

In [3]:
parsing_paths = glob.glob(os.path.join(root,'stats/1-parsing','*gov_stat.csv'))

df_parsing = pd.DataFrame()
for path in parsing_paths:
    df_parsing = pd.concat([df_parsing,pd.read_csv(path)[['dataset_name','n_bldgs']]])

In [4]:
df_parsing=df_parsing.reset_index(drop=True)


In [5]:
# abruzzo stat file was incorrect. Corrected by computing the number of buildings again from the intermediary file
df_parsing.at[16,'n_bldgs'] = 423678

In [6]:
# areas creating double counting situations, as they are available both in a country level dataset and a richer city-level dataset
df_parsing[df_parsing.dataset_name.isin(['vantaa-gov','helsinki-gov','prague-gov','brno-gov'])]

,dataset_name,n_bldgs
8,vantaa-gov,58506
13,brno-gov,112962
23,helsinki-gov,77231
28,prague-gov,225165


In [7]:
df_parsing = df_parsing[~df_parsing.dataset_name.isin(['vantaa-gov','helsinki-gov','prague-gov','brno-gov'])]

In [8]:
input_parsing = pd.read_csv(os.path.join(root,'git-eubucco/database/preprocessing/1-parsing/inputs-parsing.csv'))[['country','dataset_name']]

df_parsing = pd.merge(df_parsing,input_parsing,on='dataset_name').drop(columns='dataset_name').rename(columns={'country':'dataset_name'}).groupby('dataset_name')['n_bldgs'].sum().reset_index()

In [9]:
overview_paths = glob.glob(os.path.join(root,'stats/2-db-set-up/overview/v1','gov*.csv'))
df_overview = pd.DataFrame()
for path in overview_paths:
    tmp = pd.read_csv(path)[['n_bldgs']]
    tmp = pd.DataFrame([{'dataset_name':os.path.split(path)[-1].split('_')[1],
                         'n_bldgs':tmp.n_bldgs.sum()
                         }])
    df_overview = pd.concat([df_overview,tmp])

df = pd.merge(df_parsing,df_overview,on='dataset_name',suffixes=('_parsing','_db set up'))
df.at[13,'n_bldgs_parsing'] = 14408267 # poland building file incorrect due to powiats
df['loss_abs'] = df['n_bldgs_parsing'] - df['n_bldgs_db set up']
df['loss_per'] = round(df['loss_abs'] / df['n_bldgs_parsing'],2)

In [10]:
# negative values are due to differences in building definitions for double counted areas
df.sort_values(by='dataset_name').reset_index(drop=True)

,dataset_name,n_bldgs_parsing,n_bldgs_db set up,loss_abs,loss_per
0,austria,724487,720662,3825,0.01
1,belgium,12549458,12485254,64204,0.01
2,cyprus,564347,563861,486,0.00
3,czechia,3938967,3981273,-42306,-0.01
4,denmark,5696270,5683956,12314,0.00
5,estonia,804139,801578,2561,0.00
6,finland,5509016,5405672,103344,0.02
7,france,47852589,47814134,38455,0.00
8,germany,27991583,27956316,35267,0.00
9,italy,16232169,16026092,206077,0.01


## Germany government

In [35]:
parsing_paths = glob.glob(os.path.join(root,'stats/1-parsing','*gov_stat.csv'))

df_parsing = pd.DataFrame()
for path in parsing_paths:
    df_parsing = pd.concat([df_parsing,pd.read_csv(path)[['dataset_name','n_bldgs']]])

deu_regions = {'DE3': 'berlin-gov', 'DE4': 'brandenburg-gov', 'DE6': 'hamburg-gov', 
                 'DE8': 'mecklenburg-vorpommern-gov', 'DE9': 'niedersachsen-gov', 
                 'DEA': 'nordrhein-westfalen-gov', 'DED': 'sachsen-gov', 
                 'DEE': 'sachsen-anhalt-gov', 'DEG': 'thuerigen-gov'}

deu_parsing = df_parsing[df_parsing.dataset_name.isin(list(deu_regions.values()))].reset_index(drop=True)

In [36]:
deu_parsing

,dataset_name,n_bldgs
0,mecklenburg-vorpommern-gov,1125170
1,sachsen-gov,2066177
2,sachsen-anhalt-gov,1746686
3,brandenburg-gov,2314099
4,nordrhein-westfalen-gov,10713759
5,berlin-gov,540126
6,niedersachsen-gov,6795240
7,hamburg-gov,382339
8,thuerigen-gov,2250018


In [41]:
deu_overview = pd.read_csv('/home/nmd/Cluster/stats/2-db-set-up/overview/v1/gov_germany_overview.csv')
regions = [[deu_regions[key] for key in deu_regions.keys() if key in x] for x in deu_overview.NUTS3_ID]
deu_overview['region'] = [x[0] if x!=[] else '' for x in regions]
deu_overview = deu_overview.groupby('region')['n_bldgs'].sum().reset_index()

pd.merge(deu_parsing,deu_overview,left_on='dataset_name',right_on='region')

,dataset_name,n_bldgs_x,region,n_bldgs_y
0,mecklenburg-vorpommern-gov,1125170,mecklenburg-vorpommern-gov,1103477
1,sachsen-gov,2066177,sachsen-gov,2065236
2,sachsen-anhalt-gov,1746686,sachsen-anhalt-gov,1746534
3,brandenburg-gov,2314099,brandenburg-gov,2310863
4,nordrhein-westfalen-gov,10713759,nordrhein-westfalen-gov,10711344
5,berlin-gov,540126,berlin-gov,537979
6,niedersachsen-gov,6795240,niedersachsen-gov,6792839
7,hamburg-gov,382339,hamburg-gov,381365
8,thuerigen-gov,2250018,thuerigen-gov,2248760


## Italy government

In [11]:
parsing_paths = glob.glob(os.path.join(root,'stats/1-parsing','*gov_stat.csv'))

df_parsing = pd.DataFrame()
for path in parsing_paths:
    df_parsing = pd.concat([df_parsing,pd.read_csv(path)[['dataset_name','n_bldgs']]])

italy_regions = {'ITF1': 'abruzzo-gov', 'ITF5': 'basilicata-gov', 'ITF6': 'calabria-gov', 'ITH5': 'emilia-romagna-gov', 'ITI4': 'lazio-gov', 'ITC3': 'liguria-gov', 'ITC4': 'lombardia-gov', 'ITC1': 'piemonte-gov', 'ITG2': 'sardegna-gov', 'ITI1': 'toscana-gov', 'ITH2': 'trentino-alto-adige-gov', 'ITC2': 'valle-d-aosta-gov', 'ITH3': 'veneto-gov'}

it_parsing = df_parsing[df_parsing.dataset_name.isin(list(italy_regions.values()))].reset_index(drop=True)

In [12]:
# abruzzo
it_parsing.at[4,'n_bldgs'] = 423678

In [13]:
it_overview = pd.read_csv('/home/nmd/Cluster/stats/2-db-set-up/overview/v1/gov_italy_overview.csv')
regions = [[italy_regions[key] for key in italy_regions.keys() if key in x] for x in it_overview.NUTS3_ID]
it_overview['region'] = [x[0] if x!=[] else '' for x in regions]
it_overview = it_overview.groupby('region')['n_bldgs'].sum().reset_index()

pd.merge(it_parsing,it_overview,left_on='dataset_name',right_on='region')

,dataset_name,n_bldgs_x,region,n_bldgs_y
0,basilicata-gov,477261,basilicata-gov,476111
1,emilia-romagna-gov,1877759,emilia-romagna-gov,1876953
2,valle-d-aosta-gov,52671,valle-d-aosta-gov,50146
3,toscana-gov,1501914,toscana-gov,1500419
4,abruzzo-gov,423678,abruzzo-gov,421472
5,trentino-alto-adige-gov,160077,trentino-alto-adige-gov,159991
6,piemonte-gov,2167166,piemonte-gov,2144572
7,lazio-gov,2413601,lazio-gov,2261358
8,sardegna-gov,337366,sardegna-gov,334407
9,lombardia-gov,3349150,lombardia-gov,3341526
